In [75]:
import pandas as pd
import numpy as np
import os


In [76]:
input_data = pd.read_csv('data.csv')

In [77]:
input_data = input_data.iloc[:, :68]

In [78]:
input_data.to_csv('data.csv', index=False)
input_data.replace('FALSE', 0, inplace=True)

In [79]:
columns_to_select = [
    'Team Number', 'AC1', 'AC2', 'AC3', 
    'AC4', 'AARL', 'AARH', 'AAP', 'AAB', 'AM', 'TC1', 
    'TC2', 'TC3', 'TC4', 'TARL', 'TARH', 'TAP', 'TAB'
]

input_data = input_data[columns_to_select]

In [80]:
averaged_data = input_data.groupby("Team Number").mean().reset_index()
stdev_data = input_data.groupby("Team Number").std().reset_index()

In [81]:
# Game data for team 1

game1 = [7619,5586,4674]

game1mask = averaged_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game1
), axis=1)

game1data = averaged_data[game1mask]

# Game data for team 2

game2 = [3381,1619,9523]

game2mask = averaged_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game2
), axis=1)

game2data = averaged_data[game2mask]

# Deviations 

deviations1 = stdev_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game1
), axis=1)

deviations2 = stdev_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game1
), axis=1)


In [82]:
game1data.sum()

Team Number    17879.000000
AC1                0.666667
AC2                0.125000
AC3                0.000000
AC4                0.000000
AARL               0.000000
AARH               0.000000
AAP                0.000000
AAB                0.000000
AM                 2.152778
TC1               10.111111
TC2                0.333333
TC3                0.111111
TC4                1.333333
TARL               1.416667
TARH               1.847222
TAP                1.750000
TAB                0.000000
dtype: float64

In [83]:
game2data.sum()

Team Number    5000.000000
AC1               0.125000
AC2               0.500000
AC3               0.000000
AC4               0.625000
AARL              0.000000
AARH              0.000000
AAP               0.000000
AAB               0.000000
AM                1.652778
TC1               0.250000
TC2               1.250000
TC3               2.375000
TC4               6.000000
TARL              0.486111
TARH              1.222222
TAP               0.222222
TAB               0.222222
dtype: float64

In [86]:
# Deviation calculation
sigma = pd.Series([1, 2, 1])
deviated1 = stdev_data.mul(sigma, axis=0) + game1data
deviated2 = stdev_data.mul(sigma, axis=0) + game2data

# Coral clipping
# Every high algae takes out 2 L3 scores, and every low algae takes out 2 L3 and 2 L2 scores
l4_def = 12
l3_def = 0
l2_def = 6

def clip(input_data):
    series = input_data

    l3_cap = l3_def + (input_data["AARL"] * 2) + (input_data["AARH"] * 2) + (input_data["TARL"] * 2) + (input_data["TARH"] * 2)
    l2_cap = l2_def + (input_data["AARL"] * 2) + (input_data["TARL"] * 2)

    if (series["TC3"] > l3_cap):
        series["TC3"] = l3_cap - series["AC3"]
    
    if (series["TC2"] > l2_cap):
        series["TC2"] = l2_cap - series["AC2"]
    
    return series

clipped1 = clip(game1data.sum())
clipped2 = clip(game2data.sum())

clipped_d1 = clip(deviated1.sum())
clipped_d2 = clip(deviated2.sum())

In [89]:
dataWeights = pd.Series([
    0, 3, 4, 6, 7, 0, 0, 6, 4, 3, 2, 3, 4, 5, 0, 0, 6, 4
])


if len(clipped1) == len(dataWeights):
    weighted_sum = (clipped1.reset_index(drop=True) * dataWeights).sum()
    print(f"Points for team 1: {weighted_sum}")
else:
    print("The number of weights must match the number of values in the series.")

if len(clipped2) == len(dataWeights):
    weighted_sum = (clipped2.reset_index(drop=True) * dataWeights).sum()
    print(f"Points for team 2: {weighted_sum}")
else:
    print("The number of weights must match the number of values in the series.")

# Print deviated

if len(clipped_d1) == len(dataWeights):
    weighted_sum = (clipped_d1.reset_index(drop=True) * dataWeights).sum()
    print(f"Points 2-sigma away for team 1: {weighted_sum}")
else:
    print("The number of weights must match the number of values in the series.")

if len(clipped_d2) == len(dataWeights):
    weighted_sum = (clipped_d2.reset_index(drop=True) * dataWeights).sum()
    print(f"Points 2-sigma away for team 2: {weighted_sum}")
else:
    print("The number of weights must match the number of values in the series.")

Points for team 1: 47.791666666666664
Points for team 2: 57.68055555555555
Points 2-sigma away for team 1: 0.0
Points 2-sigma away for team 2: 82.9759116043078
